[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Etiquetado_openai.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [1]:
!pip install -q PyPDF2 python-docx openpyxl python-pptx xlrd transformers office365-rest-python-client msal requests
!pip install -q transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 6.3 MB/s eta 0:00:00


In [2]:
!apt-get install git -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [14]:
import os
import io
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader
import docx
import openpyxl
from pptx import Presentation
import xlrd
from transformers import pipeline
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import os
import requests
import msal
import csv
import getpass
from openai import OpenAI
from transformers import pipeline
import time

# Configuración

In [4]:
api_key = getpass.getpass("Introduce tu OpenAI API Key: ")
client = OpenAI(api_key=api_key)
token = getpass.getpass("Introduce tu GitHub token: ")

Introduce tu OpenAI API Key: ··········
Introduce tu GitHub token: ··········


In [32]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"  # tu nuevo Client ID
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"

In [33]:
!git config --global user.email "marcomendieta08@gmail.com"
!git config --global user.name "marcomendieta08"
!git clone https://github.com/pugapatricia/gestion-documentaria-para-pymes.git

fatal: destination path 'gestion-documentaria-para-pymes' already exists and is not an empty directory.


# Conección con OneDrive


In [40]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()

To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code L6R7BQWT to authenticate.


#Funciones

Clasificador OpenAI


In [34]:
ticketsTXT = "https://raw.githubusercontent.com/pugapatricia/gestion-documentaria-para-pymes/refs/heads/main/etiquetado/tickers.txt"
response = requests.get(ticketsTXT)
etiquetas = response.text.strip().split(", ")
print(etiquetas)

['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad', 'arqueología', 'financiero', 'contaduría', 'restitución', 'materias', 'regresión', 'extranjero', 'renta', 'análisis', 'puente', 'servicios', 'inquilino', 'optativa', 'arrendador', 'seguridad', 'notificaciones', 'vigilancia', 'depreciación', 'empresa', 'geografía', 'monográficos', 'estancia', 'balance', 'divulgación', 'autorización', 'master', 'madrid', 'interpretación', 'ingresos', 'métodos', 'fuentes', 'idiomas', 'silvopastoril', 'extranjería', 'receptor', 'proyecto', 'interés', 'apalancamiento', 'propiedad', 'incumplimiento', 'estudios', 'nie', 'datascience', 'complutense', 'mediación', 'firma', 'liquidez', 'valoracion', 'desalojo', 'compraventa', 'alquiler', 'sociales', 'jarama', 'partes', 'contrato', 'tasas', 'humanidades', 'estudiantes', 'activos', 'actividad', 'pasivos', 'financieros', 'registro', 'divulgador', 'cesión', 'poesía', 'temporada', 'documentación', 'rentabilidad', 'comisiones', 'flujodecaja', 'patri

Lector de documentos

In [9]:
def leer_pdf(contenido, limite_palabras=None):
    texto = ""
    reader = PdfReader(io.BytesIO(contenido))
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            texto += page_text + "\n"
    # Aplicar límite de palabras al final
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_docx(contenido, limite_palabras=None):
    texto = ""
    doc = docx.Document(io.BytesIO(contenido))
    # Párrafos
    for p in doc.paragraphs:
        if p.text.strip():
            texto += p.text + "\n"
    # Tablas
    for table in doc.tables:
        for row in table.rows:
            row_text = " ".join([cell.text for cell in row.cells if cell.text.strip()])
            if row_text:
                texto += row_text + "\n"
    # Aplicar límite al final
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_excel(contenido, limite_palabras=None):
    texto = ""
    wb = openpyxl.load_workbook(io.BytesIO(contenido), data_only=True, read_only=True)
    for sheet in wb.worksheets:
        for row in sheet.iter_rows(values_only=True):
            row_text = " ".join([str(cell) for cell in row if cell])
            if row_text:
                texto += row_text + "\n"
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_xls(contenido, limite_palabras=None):
    texto = ""
    temp_file = "temp.xls"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    wb = xlrd.open_workbook(temp_file)
    for sheet in wb.sheets():
        for row_idx in range(sheet.nrows):
            row = sheet.row_values(row_idx)
            row_text = " ".join([str(cell) for cell in row if cell])
            if row_text:
                texto += row_text + "\n"
    os.remove(temp_file)
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_pptx(contenido, limite_palabras=None):
    texto = ""
    temp_file = "temp.pptx"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    prs = Presentation(temp_file)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                texto += shape.text + "\n"
    os.remove(temp_file)
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_txt_csv(contenido, limite_palabras=None):
    texto = contenido.decode("utf-8", errors="ignore")
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_archivo(nombre, contenido, limite_palabras=None):
    ext = nombre.split(".")[-1].lower()
    if ext == "pdf":
        return leer_pdf(contenido, limite_palabras)
    elif ext == "docx":
        return leer_docx(contenido, limite_palabras)
    elif ext == "xlsx":
        return leer_excel(contenido, limite_palabras)
    elif ext == "xls":
        return leer_xls(contenido, limite_palabras)
    elif ext == "pptx":
        return leer_pptx(contenido, limite_palabras)
    elif ext in {"txt", "csv"}:
        return leer_txt_csv(contenido, limite_palabras)
    else:
        return ""

In [38]:
def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = re.sub(r'\d+\n', '', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^a-zA-Záéíóúüñ0-9\s,.\-()/:]', '', texto)
    return texto.strip()

def etiquetar_texto(texto, etiquetas, modelo="gpt-5-mini"):
    prompt = f"""
    Analiza el siguiente texto y selecciona entre 5 y 10 etiquetas de la lista dada
    que mejor representen los temas principales. No inventes etiquetas fuera de la lista.

    Lista de etiquetas disponibles: {', '.join(etiquetas)}

    Texto: \"\"\"{texto}\"\"\"

    Selecciona solo etiquetas que coincidan exactamente con las de la lista. No combines palabras ni inventes nuevas.
    """
    respuesta = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=400
    )
    resultado = respuesta.choices[0].message.content.strip()
    seleccionadas = [e.strip() for e in resultado.split(",")]
    if len(seleccionadas) < min_etiquetas:
        for e in etiquetas:
            if e not in seleccionadas:
                seleccionadas.append(e)
            if len(seleccionadas) >= min_etiquetas:
                break
    return seleccionadas[:max_etiquetas]

In [44]:
texto_ejemplo = "La Universidad Complutense de Madrid lanzó un proyecto en datascience para analizar el impacto ambiental en zonas silvopastoriles. Los estudiantes participan usando métodos de análisis y regresión, mientras la administración gestiona rentas, contratos, autorizaciones y la rentabilidad de los activos y pasivos. Además, se recopila documentación histórica y arqueológica para su divulgación, respetando la propiedadintelectual y la confidencialidad de los materiales."

texto_limpio = limpiar_texto(texto_ejemplo)
print("Texto limpio:", texto_limpio)

etiquetas_seleccionadas = etiquetar_texto(texto_limpio, etiquetas)
print("Etiquetas seleccionadas:", etiquetas_seleccionadas)


Texto limpio: La Universidad Complutense de Madrid lanzó un proyecto en datascience para analizar el impacto ambiental en zonas silvopastoriles. Los estudiantes participan usando métodos de análisis y regresión, mientras la administración gestiona rentas, contratos, autorizaciones y la rentabilidad de los activos y pasivos. Además, se recopila documentación histórica y arqueológica para su divulgación, respetando la propiedadintelectual y la confidencialidad de los materiales.
Etiquetas seleccionadas: ['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad']


# Leer archivos de OneDrive

In [45]:
archivos_permitidos = [
    item for item in data.get("value", [])
    if "folder" not in item and item["name"].split(".")[-1].lower() in ext_permitidas
]
archivos_a_procesar = archivos_permitidos[:5]

resultados = {}

for item in archivos_a_procesar:
    nombre = item["name"]
    download_url = item["@microsoft.graph.downloadUrl"]

    print(f"Procesando archivo: {nombre}")
    file_bytes = requests.get(download_url).content
    texto = leer_archivo(nombre, file_bytes, limite_palabras=None)
    texto_limpio = limpiar_texto(texto)
    if texto_limpio:
        etiquetas_detectadas = etiquetar_texto(texto_limpio, etiquetas)
    else:
        etiquetas_detectadas = []

    resultados[nombre] = etiquetas_detectadas

    print(f"{nombre} → {etiquetas_detectadas}")


Procesando archivo: 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf
11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf → ['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad']
Procesando archivo: 2016-admitidos_Segundo ciclo- Cursos monográficos.xls
2016-admitidos_Segundo ciclo- Cursos monográficos.xls → ['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad']
Procesando archivo: 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx
2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx → ['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad']
Procesando archivo: 660d1bfb7c43622a597a4000_Non-disclosure agreement nda template contract.pdf
660d1bfb7c43622a597a4000_Non-disclosure agreement nda template contract.pdf → ['tauromaquia', 'pasaporte', 'señal', 'medioambiente', 'universidad']
Procesando archivo: Acuerdo_no_Divulgacion_Unilateral_UE.pdf
Acuerdo_no_Divulgacio

#Guardar resultados

In [13]:
    # Guardar todos los resultados en CSV
    with open("etiquetas_onedrive.csv", "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["Archivo", "Etiquetas"])
        for nombre, etiquetas_detectadas in resultados.items():
            etiquetas_str = ", ".join(etiquetas_detectadas) if etiquetas_detectadas else "Sin etiquetas"
            writer.writerow([nombre, etiquetas_str])

    print("Resultados guardados en etiquetas_onedrive.csv")

else:
    print("No se encontraron archivos permitidos para procesar.")

SyntaxError: invalid syntax (ipython-input-3827043340.py, line 11)

## Codigo para cargar en la etiqueta de descripcion en onedrive

In [ ]:
"""
# ============================
# Configuración
# ============================
json_path = "etiquetas_onedrive.json"
headers = {"Authorization": f"Bearer {access_token}"}

# Carpeta objetivo en OneDrive
carpeta_objetivo = "Etiquetados"

# ============================
# Cargar JSON de etiquetas
# ============================
with open(json_path, "r", encoding="utf-8") as f:
    etiquetas_data = json.load(f)

# ============================
# Aplicar etiquetas en la descripción
# ============================
for archivo, etiquetas in etiquetas_data.items():
    if not etiquetas:
        continue

    # Buscar archivo en OneDrive
    url_file = f"https://graph.microsoft.com/v1.0/me/drive/root:/{carpeta_objetivo}/{archivo}"
    resp_file = requests.get(url_file, headers=headers)

    if resp_file.status_code != 200:
        print(f"⚠️ No se encontró {archivo} en OneDrive")
        continue

    file_id = resp_file.json()["id"]

    # Guardamos etiquetas en el campo "description"
    url_update = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}"
    payload = {"description": ", ".join(etiquetas)}

    response_update = requests.patch(
        url_update,
        headers={**headers, "Content-Type": "application/json"},
        json=payload
    )

    if response_update.status_code in [200, 204]:
        print(f"✅ Etiquetas {etiquetas} aplicadas a {archivo} en la descripción")
    else:
        print(f"⚠️ Error al actualizar {archivo}: {response_update.text}")
"""